### (1) Install Python Packages (Run Once)

Run the section below to install needed Python packages

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### (2) Edit Tool Configuration and Execute

In [1]:
from run_stages.pattern_generation_stage import Text, Grating, Rectangle, Circle, FullField
### CONFIGURATION
configuration = {}

# geometry-defined configuration
configuration["model"]                            = "bipolar"     # model geometry: monopolar or bipolar
configuration["pixel_size"]                       = 100           # pixel size
configuration["frame_width"]                      = 1000          # implant radius in mm
configuration["geometry"]                         = "Flat_human"  # geometry settings: HC/Pillar/Flat devices in pdish/rat/human
configuration["number_of_diodes"]                 = 2             # number of photo diodes per pixel
configuration["sirof_capacitance"]                = 6             # SIROF capacitance in mF/cm^2
configuration["photosensitive_area_edge_to_edge"] = 92            # edge-to-edge size of the photosensitive area
configuration["active_electrode_radius"]          = 17            # radius of the active electrode in um
configuration["light_to_current_conversion_rate"] = 0.4           # light to current conversion rate in A/W
configuration["photosensitive_area"]              = 4075.72       # total photosensitive area in um^2. Assign "None" for auto calculation

# R matrix parameters
configuration["r_matrix_output_file"]             = f'R_{configuration["geometry"]}_PS{configuration["pixel_size"]}.pkl'
configuration["r_matrix_conductivity"]            = 1             # conductivity scaling factor of the electrolyte

# dynamic simulation configuration
configuration["Ipho_scaling"]                     = 1             # artificial scaling of photo current, useful for parametric sweep (e.g. S-D curve)
configuration["Isat"]                             = 0.02          # diode saturation current in pA
configuration["ideality_factor"]                  = 1.14          # ideality factor n of the diode
configuration["shunt_resistance"]                 = 790.15E3      # shunt resistance in Ohm. Assign "None" if no shunt
configuration["initial_Vactive"]                  = .4            # Initial bias of the active electrode in V
configuration["temperature"]                      = 37
configuration["nominal_temperature"]              = 25
configuration["simulation_duration_sec"]          = 0.5           # simulation duration in seconds
configuration["simulation_resolution_ms"]         = None          # None defaults to Xyce inner value

# input paths
configuration["user_files_path"]                  = None          # If set to None, defaults to inner user_files directory

# input pattern - if set to None, enter the parameters of the desired pattern to be generated
configuration["video_sequence_name"]              = ["input_sample_3"]
configuration["generate_pattern"]                 = True
img_seq = {}
img_seq["script"]                                 = [
                                                      ["Light Intensity", 1.0, "mW/mm^2"],
                                                      ["Frame period", 100, "ms"],
                                                      ["Time step", 0.05, "ms"]
                                                    ]

img_seq["list_image_names"]                       = ["Text_shapes", "Grating"]             # The number of images in the GIF sequence
img_seq["Text_shapes"]                            = [ ("Number_repetitions", 2),
                                                      # Name, exposure time in ms, pattern configuration  
                                                      ("Subframe1", 15, [Text(user_position=(0, 0), rotation=0, text="C", letter_size=5)]),
                                                      ("Subframe2", 15, [Rectangle(user_position=(0, 5), rotation=45, width=100, height=100)]),
                                                      ("Subframe3", 15, [Circle(user_position=(0, - 5.5), diameter=200)]),
                                                      ("Subframe4", 30, [ Text((-5, 0), 90, "THIS IS \n\n A SIMPLE TEST.", 1),
                                                                          Text((5.5, 0), -90, "THIS IS \nA SIMPLE TEST.", 1) 
                                                                        ]),
                                                      ("Subframe6", 25, [FullField("black")]) 
                                                    ]
img_seq["Grating"]                                = [ ("Number_repetition", 1),
                                                      # Name, exposure time in ms, pattern configuration 
                                                      ("Subframe1", 75, [Grating(user_position=(1.5, 0), rotation = 45, width_grating= 75, pitch_grating= 75)]),
                                                      ("Subframe2", 25, [FullField("black")])
                                                    ]
configuration["patterns_config"]                  = img_seq

configuration["pixel_label_input_file"]           = f'image_sequence/pixel_label_PS{configuration["pixel_size"]}-lg.pkl'

# define input files for monopolar arrays
configuration["monopolar"] = \
    {
    "return_to_active_area_ratio": 1.465123,                      # ratio between return area and total active
        # area
    "r_matrix_input_file_diagonal": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_EP_self.csv',
    "r_matrix_input_file_non_diagonal": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_EP_Rmat.csv',
    "r_matrix_simp_ratio": 0.1
    }

# define input files for bipolar arrays
configuration["bipolar"] = \
    {
    "additional_edges": 142,                                     # bipolar only: edge segments of the return
    "r_matrix_input_file_active": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_active.csv',
    "r_matrix_input_file_return": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_return.csv',
    "r_matrix_input_file_return_neighbor": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_return_neighbor.csv',
    "r_matrix_input_file_return_near": f'r_matrix/COMSOL_results/{configuration["geometry"]}_PS{configuration["pixel_size"]}_UCD_return_near.csv',
    "r_matrix_input_file_px_pos": f'r_matrix/COMSOL_results/PS{configuration["pixel_size"]}-lg_pos.csv'
    }

# post-process parameters
configuration["post_process"] = \
    {
    "stimulation_duration_in_ms": 9.8,
    "analyze_time_dynamics": False,
    "pulse_start_time_in_ms": 200,
    "depth_values_in_um": None                                      # if None, defaults to all available in model
    }


### EXECUTION
from RPSim import run_rpsim
run_stages = None
run_rpsim(configuration=configuration, run_stages=run_stages)

[2023-10-18 15:05:06] INFO       run_rpsim        Staring a new run
[2023-10-18 15:05:06] INFO       run_rpsim        Output directory: /Users/quentindevaud/Desktop/EPFL/Master/MA5/Computational/RPSim/user_files/user_output/15_05_06-2023_10_18
[2023-10-18 15:05:06] INFO       run_rpsim        Requested run stages: ['resistive_mesh', 'current_sequence', 'circuit', 'simulation', 'post_process', 'plot_results']
[2023-10-18 15:05:06] INFO       run_rpsim        Running the following configuration
+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| model  | bipolar                                                                                                                                                          |
| pixel_ | 100                                                                                                                                            